# Spark Streaming con Dstreams
## Ejercicio 3
> Contar palabras usando ventanas y un directorio de checkpoints.

- Creamos una carpeta en el directorio desde donde lanzamos los notebooks llamada *checkpointDirectory*.

- Comenzamos creando StreamingContext a partir del SparkContext, para ello, definimos una función:

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
checkpointDir = './checkpointDirectory'

In [2]:
def createContext():
    sc = SparkContext("local[2]", "WordCount")
    ssc = StreamingContext(sc, 5)
    ssc.checkpoint(checkpointDir)
    return ssc

In [3]:
ssc = StreamingContext.getOrCreate(checkpointPath = checkpointDir, setupFunc = createContext)

- Leemos las líneas y las separamos en palabras como en los ejercicios anteriores:

In [4]:
lines = ssc.socketTextStream("localhost", 9999)
words =lines.flatMap(lambda line:line.split(" "))

- Creamos un par para cada palabra:

In [5]:
pairs = words.map(lambda word:(word, 1))

- Reducimos los datos por clave y por una ventana de 30 segundos con desplazamiento de 10 segundos:

In [6]:
windowedWordCounts = pairs.reduceByKeyAndWindow(lambda x,y: x+y, lambda x,y: x-y, 30, 10)

- Mostramos los elementos por consola:

In [ ]:
windowedWordCounts.pprint()

ssc.start()
ssc.awaitTermination(60)